#En este notebook aplicaremos un algoritmo de clasificación realizando:

1.   Pre procesamiento.
2.   Entrenamiento del algoritmo
3.   Testeo del algoritmo.
4.   Cálculo de las métricas de rendimiento.



In [ ]:
import numpy as np
import pandas as pd

##Información del dataset:
[AQUí](https://www.kaggle.com/shrutimechlearn/churn-modelling)

In [ ]:
!wget https://www.dropbox.com/s/rv4ubf4htkia1af/3.4.3%20Churn_Modelling.csv

--2022-05-04 20:46:35--  https://www.dropbox.com/s/rv4ubf4htkia1af/3.4.3%20Churn_Modelling.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/rv4ubf4htkia1af/3.4.3%20Churn_Modelling.csv [following]
--2022-05-04 20:46:35--  https://www.dropbox.com/s/raw/rv4ubf4htkia1af/3.4.3%20Churn_Modelling.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbda61ea673023318d3bfcb2746.dl.dropboxusercontent.com/cd/0/inline/BkrqrgKjkWsqcf8WTw00UsqxNei2tOG2AzuwJ857PnqmPBZnPHnYKXb8KD35K0QRp5EkOosuOf8Uuxwvf0fEQS-LrRLtDn8Z7EZh0EZB_Ez-71VNyB0nR5ex8Ucs2WczYZA2pn-MpK2OG-a8b8g0MVclHadb6bSzBy3Fi6D1uD9KaA/file# [following]
--2022-05-04 20:46:36--  https://ucbda61ea673023318d3bfcb2746.dl.dropboxusercontent.com/cd/0/inline/BkrqrgKjkWsqcf8WTw00Usq

In [ ]:
# Cargamos los datos
df1 = pd.read_csv("3.4.3 Churn_Modelling.csv", sep=",")

In [ ]:
df1

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
df1.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [ ]:
# Extraemos la variable que vamos a estimar.
Y = df1['Exited']
Y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [ ]:
# Eliminamos variables que no entregan información importante.
X = df1.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Exited'])
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Creamos el objeto
ohe = OneHotEncoder()
# Tomamos la variable a encodear
geo = np.array(X.Geography)
# Transforma como vector fila
geo2 = geo.reshape(-1,1)
# Creamos el modelo de encodeo
ohe.fit(geo2)
# Aplicamos el encodeo
geo3 = ohe.transform(geo2).toarray()
geo3

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [ ]:
# concatenar geo3 con X
X = pd.concat([X, pd.DataFrame(geo3)], axis=1)
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,0,1,2
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0


In [ ]:
# cambiamos nombres de las variables creadas.
X.rename(columns={0: 'France', 1: 'Germany', 2:'Spain'}, inplace=True)
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0


In [ ]:
X = X.drop(columns = ['Geography'])
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain
0,619,Female,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,Female,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,Female,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,Female,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,Male,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0
9996,516,Male,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0
9997,709,Female,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0
9998,772,Male,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0


Hacemos lo mismo con la variable genero

Intento hacerlo usted ahora.

In [ ]:
ohe = OneHotEncoder()
gen = np.array(X.Gender)
gen2 = gen.reshape(-1,1)
ohe.fit(gen2)
gen3 = ohe.transform(gen2).toarray()
gen3

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [ ]:
X = pd.concat([X, pd.DataFrame(gen3)], axis=1)
X.rename(columns={0: 'Female', 1: 'Male'}, inplace=True)
X = X.drop(columns = ['Gender'])
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,France,Germany,Spain,Female,Male
0,619,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0,1.0,0.0
1,608,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0,1.0,0.0
2,502,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0,1.0,0.0
3,699,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0,1.0,0.0
4,850,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0,0.0,1.0
9996,516,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0,0.0,1.0
9997,709,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0,1.0,0.0
9998,772,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0,0.0,1.0


# Ahora tenemos todas las variables numéricas. Podriamos pasar a la tarea de clasificación con el algoritmo K-NN.

##¿Haría algo más a los datos?

In [ ]:
# importamos la función que divide el train y el test
from sklearn.model_selection import train_test_split

In [ ]:
# dividimos los datos entre training y testing set.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=17)

In [ ]:
# importamos el algoritmo KNN
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# creamos el objeto
model = KNeighborsClassifier()

In [ ]:
# entrenamos el modelo
model.fit(X_train, Y_train)

KNeighborsClassifier()

In [ ]:
# realizamos predicciones de los datos de test
y_pred = model.predict(X_test)

In [ ]:
# importamos las funciones que nos permiten calcular las métricas de rendimiento
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(confusion_matrix(Y_test, y_pred))

[[2235  152]
 [ 560   53]]


##¿Qué podemos decir del resultado?

In [ ]:
tn, fp, fn, tp = confusion_matrix(Y_test, y_pred).ravel()
print(tn, fp, fn, tp)

2235 152 560 53


Recordando la matriz de confusión

![](https://static.packt-cdn.com/products/9781838555078/graphics/C13314_06_05.jpg)

In [ ]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.94      0.86      2387
           1       0.26      0.09      0.13       613

    accuracy                           0.76      3000
   macro avg       0.53      0.51      0.50      3000
weighted avg       0.69      0.76      0.71      3000



In [ ]:
total = tn+fp+fn+tp
errores = (fp+fn)
aciertos = (tn+tp)
print("Total =", total)
print("Cantidad de errores =", errores)
print("Cantidad de aciertos =", aciertos)
print("% de errores = {:.2f}%".format(100*errores/total))
print("% de aciertos = {:.2f}%".format(100*aciertos/total))

Total = 3000
Cantidad de errores = 712
Cantidad de aciertos = 2288
% de errores = 23.73%
% de aciertos = 76.27%


##¿Qué podemos decir del resultado?

***Respuesta***
+ Se equivoca 712 veces de un total de 3000 obervaciones lo que representa, aproximadamente, el 24%
+ Acierta 2288 veces de un tota del 3000 observaciones lo que representa, aproximadamente, el 76%

##También podemos ver como le fue al algoritmo con el training set.
## Aquí calculamos las mismas métricas

In [ ]:
Y_train_predicted = model.predict(X_train)

In [ ]:
print(confusion_matrix(Y_train, Y_train_predicted))

[[5404  172]
 [1128  296]]


In [ ]:
print(classification_report(Y_train, Y_train_predicted))

              precision    recall  f1-score   support

           0       0.83      0.97      0.89      5576
           1       0.63      0.21      0.31      1424

    accuracy                           0.81      7000
   macro avg       0.73      0.59      0.60      7000
weighted avg       0.79      0.81      0.77      7000



##¿Son esperables los resultados?


Nos falta calcular al área bajo la curva (ROC AUC)

In [ ]:
# importamos la función
from sklearn.metrics import roc_auc_score

In [ ]:
# para el testing set
roc_auc_score(Y_test, y_pred)

0.5113908876998915

In [ ]:
# para el training set
roc_auc_score(Y_train, Y_train_predicted)

0.588509341801944

# Acerca de la curva de ROC

La curva ROC indica qué tan bueno puede distinguir el modelo entre dos cosas, por ejemplo, si un paciente tiene cáncer o no. Mejores modelos pueden distinguir con precisión entre los dos, mientras que un modelo pobre tendrá dificultades para distinguir entre los dos.


Veamos un ejemplo:

![](https://live.staticflickr.com/65535/48048973513_1f9d124b54_b.jpg)

## Recordar los conceptos de sensibilidad y especificidad

La ***sensibilidad o recall***, es la proporción de pacientes que se identificaron correctamente por tener cáncer, es decir, verdadero positivo, sobre el número total de pacientes que realmente tienen la enfermedad.

Por su parte, ***especificidad*** es la proporción de pacientes que se identificaron correctamente por no tener cáncer, verdadero positivo, sobre el número total de pacientes que no tienen la enfermedad.

**Fórmulas**

$sensibilidad = \frac{tp}{(tp+fn)}$

$especificidad = \frac{tn}{(tn+fp)}$




Si volvemos a la gráfica anterior, si se disminuye el valor del umbral, se obtienen más valores negativos, aumentando la sensibilidad, pero disminuyendo la especificidad.

![](https://live.staticflickr.com/65535/48048973193_8246b103b7_c.jpg)

## Área bajo la curva

El AUC es el área bajo la curva ROC. Este puntaje nos da una buena idea de qué tan bien funciona el modelo.

Este método es conveniente por las siguientes razones:

+ Es invariable con respecto a la escala, mide qué tan bien se clasifican las predicciones, en lugar de sus valores absolutos.
+ Es invariable con respecto al umbral de clasificación, mide la calidad de las predicciones del modelo, sin tener en cuenta qué umbral de clasificación se elige.

Algunos ejemplos de esto:

### Ejemplo 1

![](https://live.staticflickr.com/65535/48049010727_ef276cb443_b.jpg)

Esta es una situación ideal. Cuando dos curvas no se superponen en absoluto, el modelo tiene una medida ideal de separación. Es perfectamente capaz de distinguir entre clase positiva y clase negativa.

### Ejemplo 2

![](https://live.staticflickr.com/65535/48048972478_80ec37664c_b.jpg)

Cuando dos distribuciones se superponen, introducimos errores. Dependiendo del umbral, podemos minimizarlos o maximizarlos. Cuando AUC es 0.7, significa que hay 70% de probabilidad de que el modelo pueda distinguir entre clase positiva y clase negativa.

### Ejemplo 3

![](https://live.staticflickr.com/65535/48049010402_f5ff692cb6_b.jpg)

Esta es la **peor situación.** Cuando el AUC es aproximadamente 0.5, el modelo
no tiene capacidad de discriminación para distinguir entre clase positiva y
clase negativa.

### Ejemplo 4

![](https://live.staticflickr.com/65535/48048932026_19ee696fdb_b.jpg)

Cuando AUC es aproximadamente 0, el modelo en realidad está correspondiendo las clases. Significa que el modelo predice la clase negativa como una clase positiva y viceversa.

## Notas acerca de KNN

+ El [algoritmo]((https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier) por defecto usa un valor de K igual a 5.
+ Un valor de K grande aumenta la probabilidad de underfitting
+ Un valor de K pequeño aumenta la probabilidad de overfitting

## Comprobación del algunas métricas

In [ ]:
from sklearn.metrics import recall_score

In [ ]:
recall_manual = tp/(tp+fn)
print("Recall manual : {:.4f}".format(recall_manual))
print("Recall : {:.4f}".format(recall_score(Y_test, y_pred)))

Recall manual : 0.0865
Recall : 0.0865
